# Averaging models
Probably the best way to ensure robust inferences and estimate errors is to have multiple experimental replicates, ideally on different libraries.

Here we describe how to average model fits across libraries and/or replicates.

## Split data into replicates
We will use our data for the RBD as an earlier examples, but split it into several libraries / replicates.

Specifically, we will fit two different libraries: `avg2muts` and `avg3muts`, which have different barcodes and also different mutation rates (although of course in real life you might sometimes want to average results from different libraries with the same mutation rates).
We will also simulate having two replicates for each library just by sampling each library.
To make this example faster, we'll just use one concentration:

In [1]:
import pandas as pd

import polyclonal.polyclonal
import polyclonal.polyclonal_collection


# read data
all_data = pd.read_csv("RBD_variants_escape_noisy.csv", na_filter=None)

# split by library and replicates
libraries = ["avg2muts", "avg3muts"]  # the two libraries to use
concentrations = [1]  # use just use this concentration
n_replicates = 2  # number of replicates per library

data_by_replicate = {
    (library, replicate + 1): (
        all_data.query("library == @library")
        .query("concentration in @concentrations")
        .sample(frac=0.3, random_state=replicate)
    )
    for library in libraries
    for replicate in range(n_replicates)
}

## Fit models to each replicate
We now fit a `Polyclonal` model to each replicate using just 2 epitopes, as the data don't seem sufficient to accurately fit all three epitopes.
Then we arrange the models in a data frame:

In [2]:
# first create a data frame with all the models
models_by_replicate = {}
for (library, replicate), data in data_by_replicate.items():
    model = polyclonal.Polyclonal(data_to_fit=data, n_epitopes=2)
    models_by_replicate[(library, replicate)] = model
models_df = (
    pd.Series(models_by_replicate, name="model")
    .rename_axis(["library", "replicate"])
    .reset_index()
)

# now fit the models
n_fit, n_failed, models_df["model"] = polyclonal.polyclonal_collection.fit_models(
    models_df["model"],
    n_threads=2,
    reg_escape_weight=0.01,
    reg_uniqueness2_weight=0,
)

Note how the models are arranged in a data frame:

In [3]:
# NBVAL_IGNORE_OUTPUT

models_df

,library,replicate,model
0,avg2muts,1,<polyclonal.polyclonal.Polyclonal object at 0x...
1,avg2muts,2,<polyclonal.polyclonal.Polyclonal object at 0x...
2,avg3muts,1,<polyclonal.polyclonal.Polyclonal object at 0x...
3,avg3muts,2,<polyclonal.polyclonal.Polyclonal object at 0x...


## Average the models
Now we create a `PolyclonalAverage` model with the models to average.
Note that by default the "average" used by `PolyclonalAverage` is the **median** rather than the **mean** between epitopes, although this is a parameter that can also be set to mean.

If your epitopes are too different or poorly defined (e.g., you are trying to fit more epitopes than can be consistently inferred from the data), then you may get an epitope harmonization error:

In [4]:
model_avg = polyclonal.PolyclonalAverage(models_df)

Let's look at the correlation among the escape at each epitope across models:

In [5]:
# NBVAL_IGNORE_OUTPUT

model_avg.mut_escape_corr_heatmap()

alt.Chart(...)

Look at the activities of the epitopes and the rest of the curves.
Note how a dark line is shown for the average, and thin lines for individual replicates.
It should generally be the case that the epitope with greater activity (more left shifted in plot below) should also be better correlated among replicates (heatmap above) as it can be inferred more reliably:

In [6]:
# NBVAL_IGNORE_OUTPUT

model_avg.curves_plot()

alt.Chart(...)

We can access the average escape values.
Note that there is also a column *escape_min_magnitude* that gives the lowest magnitude value:

In [7]:
# NBVAL_IGNORE_OUTPUT

model_avg.mut_escape_df

,epitope,site,wildtype,mutant,mutation,escape_mean,escape_median,escape_min_magnitude,escape_std,n_models,times_seen,frac_models
0,1,331,N,A,N331A,-0.067564,-0.030975,0.021971,0.328544,4,17.75,1.0
1,1,331,N,D,N331D,-0.090326,-0.083941,-0.074902,0.020104,4,11.25,1.0
2,1,331,N,E,N331E,-0.066548,-0.009506,-0.001983,0.119161,4,10.25,1.0
3,1,331,N,F,N331F,0.283373,0.192162,0.018695,0.333196,4,10.00,1.0
4,1,331,N,G,N331G,0.193667,0.177908,-0.026998,0.221850,4,25.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3859,2,531,T,R,T531R,-0.174502,-0.097747,-0.026822,0.210255,4,27.00,1.0
3860,2,531,T,S,T531S,-0.082028,-0.014567,0.005165,0.148563,4,31.75,1.0
3861,2,531,T,V,T531V,-0.042999,-0.007213,0.018650,0.103766,4,19.50,1.0
3862,2,531,T,W,T531W,0.163447,0.088392,-0.005205,0.276730,4,5.25,1.0


We can do the same for the ICXX values:

In [8]:
# NBVAL_IGNORE_OUTPUT

model_avg.mut_icXX_df(
    x=0.9,
    icXX_col="IC90",
    log_fold_change_icXX_col="log2 fold change IC90",
)

,site,wildtype,mutant,log2 fold change IC90 mean,log2 fold change IC90 median,log2 fold change IC90 min_magnitude,log2 fold change IC90 std,n_models,times_seen,frac_models
0,331,N,A,-0.164860,-0.194622,-0.119023,0.278079,4,17.75,1.0
1,331,N,D,-0.094874,-0.051912,-0.011410,0.115779,4,11.25,1.0
2,331,N,E,0.032633,0.015820,0.011409,0.059330,4,10.25,1.0
3,331,N,F,0.098268,0.139888,0.115545,0.286302,4,10.00,1.0
4,331,N,G,0.294765,0.283074,0.122967,0.156693,4,25.00,1.0
...,...,...,...,...,...,...,...,...,...,...
2100,531,T,S,-0.069089,-0.020452,-0.012045,0.138868,4,31.75,1.0
2101,531,T,T,0.000000,0.000000,0.000000,0.000000,4,NaN,1.0
2102,531,T,V,0.008317,-0.010543,0.031919,0.243751,4,19.50,1.0
2103,531,T,W,0.193380,0.109259,-0.008928,0.292706,4,5.25,1.0


Or the per-replicate escape values:

In [9]:
# NBVAL_IGNORE_OUTPUT

model_avg.mut_escape_df_replicates

,epitope,site,wildtype,mutant,mutation,escape,times_seen,library,replicate
0,1,331,N,A,N331A,-0.083922,19,avg2muts,1
1,1,331,N,D,N331D,-0.076938,10,avg2muts,1
2,1,331,N,E,N331E,-0.007855,11,avg2muts,1
3,1,331,N,F,N331F,0.039737,10,avg2muts,1
4,1,331,N,G,N331G,0.045873,18,avg2muts,1
...,...,...,...,...,...,...,...,...,...
15429,2,531,T,R,T531R,-0.033279,24,avg3muts,2
15430,2,531,T,S,T531S,-0.005251,42,avg3muts,2
15431,2,531,T,V,T531V,-0.033077,24,avg3muts,2
15432,2,531,T,W,T531W,0.546467,6,avg3muts,2


Now let's plot the escape.
See how you can select mutations based not only on how many times they are seen (averaged over all models in average), but also the number of models in which they are seen.

In [10]:
# NBVAL_IGNORE_OUTPUT

model_avg.mut_escape_plot(addtl_slider_stats={"times_seen": 2})

alt.VConcatChart(...)

Here is the same plot plotting the lowest magnitude escape value across libraries for each mutation:

In [11]:
# NBVAL_IGNORE_OUTPUT

model_avg.mut_escape_plot(
    addtl_slider_stats={"times_seen": 2}, avg_type="min_magnitude"
)

alt.VConcatChart(...)

We can also plot the log fold change in IC90 caused by each mutation:

In [12]:
# NBVAL_IGNORE_OUTPUT

model_avg.mut_icXX_plot(
    addtl_slider_stats={"times_seen": 2},
    avg_type="median",
)

alt.VConcatChart(...)

## Escape values by region
In some cases, you may want to only get the escape values for a specific region of the protein for each model being averaged.
For instance, this may be the case if you covered half the protein in one library and the other half in the other library.

There are functions to get the values by region.
Specfically:

In [13]:
model_avg.models